<a href="https://colab.research.google.com/github/JorgeLMF/cc42_tf_202014828_201915747_201921559_201811355/blob/main/Trabajo_Final_Elena_Salcedo_Nearest_Neighbor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ALGORITMO ELENA SALCEDO

##IMPORTS

In [3]:
import graphviz as gv
import numpy as np
import pandas as pd
import heapq as hq
import time
import math

In [4]:
url = "https://raw.githubusercontent.com/JorgeLMF/cc42_tf_202014828_201915747_201921559_201811355_201714378/main/DATASET/almacenesXyY.csv"
url2 = "https://raw.githubusercontent.com/JorgeLMF/cc42_tf_202014828_201915747_201921559_201811355_201714378/main/DATASET/puntosentregaXyY.csv"
file_almacenes = pd.read_csv(url).to_numpy()
pts_entrega = pd.read_csv(url2).to_numpy()

##ELABORACIÓN

###LA_MATRIX

In [5]:
# CREACIÓN DE LA MATRIZ



## Inicializar matriz
la_matrix = []


## Asignar tamaño 1000 x 1000 y llenarlo con 0s.
for i in range(1000):
    l = [0]*1000                          # N°0 -> ni almacén ni punto de entrega
    la_matrix.append(l)


## Asignarle el 1 a los puntos de la matriz que sean almacenes
for line in file_almacenes:
    la_matrix[line[0]][line[1]] = 1       # N°1 -> almacenes
    

## Asignarle el 2 a los puntos de la matriz que sean puntos de entrega
for line in pts_entrega:
    if la_matrix[line[0]][line[1]] != 1:
       la_matrix[line[0]][line[1]] = 2    # N°2 -> puntos de entrega

In [6]:
## Crear función para mostrar la matriz
def mostrar_matriz():
  for line in la_matrix:
    print(line)

###PUNTOS

In [7]:
# DARLE SUS CARACTERÍSTICAS A CADA PUNTO



## Crear una lista de puntos
puntos = list()
n = 1000

## Asignarle a cada punto en la lista una lista de 4 espacios:
for i in range(n*n):
    l = [0]*4
    puntos.append(l)

## El valor que guarda cada sublista se define a continuación
for i in range(n):
  for j in range(n):
    puntos[j+n*i][0] = j                # 0 = eje x
    puntos[j+n*i][1] = i                # 1 = eje y
    puntos[j+n*i][2] = la_matrix[i][j]  # 2 = tipo de punto
    puntos[j+n*i][3] = -1               # 3 = padre

###GRAFO

In [8]:
# CREACIÓN DEL GRAFO



## Inicializar el grafo
grafo = list()

## El grafo presentará las uniones de cada punto con:
## su punto de arriba, izquierda, derecha y abajo respectivamente, si los tuviera
for i in range(n):
  for j in range(n):
    l = list()
    if j+n*i-1000 >= 0:
      l.append(j+n*i-1000)
    if j+n*i-1 >= 0:
      l.append(j+n*i-1)
    if j+n*i+1 <= 999999:
      l.append(j+n*i+1)
    if j+n*i+1000 <= 999999:
      l.append(j+n*i+1000)
    
    grafo.append(l)

###NEAREST NEIGHBOR

In [9]:
## Inicializar listas de puntos
pts_almacenes = []
pts_entregar = []


## Insertar en cada lista los índices de los puntos que son almacenes y que son puntos de entrega, respectivamente
for i in range(n*n):
  if puntos[i][2] == 1:
    pts_almacenes.append(i)
  elif puntos[i][2] == 2:
    pts_entregar.append(i)

In [10]:
# CREACIÓN DE CLUSTERS (Sectores cuyo criterio es reunir a cada punto de entrega con su almacén más cercano)

def clusters(warehouse_points, delivery_points): # La funcion clusters pone los puntos de entrega al punto de almacenes más cercano  
  manhattan = lambda a,b: abs(puntos[a][0]-puntos[b][0])+abs(puntos[a][1]-puntos[b][1])
  cl = []
  for i in range(len(warehouse_points)):
    cl.append([])

  for j in range(len(delivery_points)):
    d = delivery_points[j]
    mn = math.inf
    idmn = 0
    for i in range(len(warehouse_points)):
      w = warehouse_points[i]
      dis = manhattan(d,w) #se obtiene la distancia de cada warehouse_points hacia las los delivery_points
      if(dis < mn):
        mn = dis
        idmn = i
    
    cl[idmn].append(d)
  
  return cl

In [11]:
# ASIGNACIÓN DE LOS CLUSTERS
clusters_ = clusters(pts_almacenes,pts_entregar)

In [12]:
# CREACIÓN DE ALGORITMO NEAREST NEIGHBOR


## Al algoritmo se le pasa como parámetro la variable que contiene a los clusters y el número de cluster al que se le desea aplicar el algoritmo
## (el número de cada cluster es también el número de orden del almacén dentro de la lista de puntos de almacenes)
def NearestNeighbor(clst,orden_pnt_almacen):

  ## se obtiene el número de punto del almacen del cluster
  new = pts_almacenes[orden_pnt_almacen]

  ## Se crea una iteración para que cada punto del cluster pueda hallar su punto más cercano en el orden en el que le toca según son escogidos
  for i in range(len(clst[orden_pnt_almacen])):

    ## Se inicializa la distancia mínima entre cada punto de entrega con el punto de almacén como infinita
    minima_distancia = math.inf

    ## Para cada punto de entrega en el cluster, si no tiene padre (no ha sido añadido antes al camino)
    ## y además es el punto más cercano a la última parada (último punto añadido al actual camino)
    ## dicho punto se añade al camino asignándole como padre al último punto añadido y volviéndose él mismo el último punto del camino
    for punto_entrega in clst[orden_pnt_almacen]:
      if puntos[punto_entrega][3]==-1:
        distancia = abs(puntos[punto_entrega][0]-puntos[new][0])+abs(puntos[punto_entrega][1]-puntos[new][1])

        if distancia < minima_distancia:
          minima_distancia = distancia
          ultimo_punto_entrega = punto_entrega

    puntos[ultimo_punto_entrega][3] = new
    new = ultimo_punto_entrega

In [13]:
## Realizar el algorimto NEAREST NEIGHBOR a todos los clusters del 51 al 75

for cluster in range(int(len(clusters_)/2+1),int(len(clusters_)*3/4)): #DEL CLUSTER 51 AL 75
  NearestNeighbor(clusters_,cluster)

###RESULTADO FINAL

In [14]:
## Mostrar las características de los puntos de entrega de los clusters del 51 al 75 ahora que el algoritmo ha sido aplicado

print(" PUNTO\t   X\tY  TIPO PADRE")
for i in range(int(len(clusters_)/2+1),int(len(clusters_)*3/4)): # INFO DE PUNTOS DE ENTREGA DE CLUSTERS 51 A 75
  for point in clusters_[i]:
    print(point,":",puntos[point])

 PUNTO	   X	Y  TIPO PADRE
501588 : [588, 501, 2, 524588]
501613 : [613, 501, 2, 501588]
506552 : [552, 506, 2, 509571]
509571 : [571, 509, 2, 514574]
513549 : [549, 513, 2, 506552]
520614 : [614, 520, 2, 501613]
524588 : [588, 524, 2, 536576]
532543 : [543, 532, 2, 532546]
532546 : [546, 532, 2, 513549]
536576 : [576, 536, 2, 546561]
540596 : [596, 540, 2, 545598]
542617 : [617, 542, 2, 520614]
545598 : [598, 545, 2, 555602]
546561 : [561, 546, 2, 532543]
554611 : [611, 554, 2, 554616]
554616 : [616, 554, 2, 542617]
555602 : [602, 555, 2, 554611]
562589 : [589, 562, 2, 564594]
564594 : [594, 564, 2, 540596]
564613 : [613, 564, 2, 570612]
565582 : [582, 565, 2, 562589]
570612 : [612, 570, 2, 574597]
570622 : [622, 570, 2, 576615]
573558 : [558, 573, 2, 570622]
574597 : [597, 574, 2, 565582]
576615 : [615, 576, 2, 564613]
600564 : [564, 600, 2, 573558]
465093 : [93, 465, 2, 477089]
469079 : [79, 469, 2, 465093]
477089 : [89, 477, 2, 485084]
477120 : [120, 477, 2, 489100]
482131 : [131, 4

###DATOS EXTRA

In [15]:
cantidades = []
for cluster in clusters_:
  cantidades.append(len(cluster))

In [16]:
from statistics import mean
from statistics import median
from statistics import mode

In [17]:
## Características respecto a la cantidad de puntos de entrega en cada Cluster

print("MAX:",max(cantidades))
print("MIN:",min(cantidades))
print("MEDIA:",mean(cantidades))
print("MEDIANA:",median(cantidades))
print("MODA:",mode(cantidades))

MAX: 123
MIN: 4
MEDIA: 34.95
MEDIANA: 31.0
MODA: 29
